In [ ]:
!pip install mlxtend

In [26]:
# Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer,mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor,BaggingRegressor
from sklearn.linear_model import LinearRegression
import math
from mlxtend.regressor import StackingRegressor
import warnings
warnings.filterwarnings("ignore")

In [35]:
#Loading the house prices dataset for the mini-challenge
df = pd.read_csv('graduate-admissions/Admission_Predict.csv')
df=df.rename(columns = {'Chance of Admit ':'Chance of Admit'})
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


# Ensemble Methods

<img src="../images/icon/ppt-icons.png" alt="Concept-Alert" style="width: 100px;float:left; margin-right:15px"/>
<br /> 
##  Mini-Challenge - 1
***
### Instructions
* Split the dataset into features and target variable(`Chance of Admit`) then split it into train and test.

In [36]:
X = df.drop('Chance of Admit',axis=1)
y = df['Chance of Admit'].copy()
X_train,X_test,y_train,y_test = train_test_split(X,y)

<img src="../images/icon/ppt-icons.png" alt="Concept-Alert" style="width: 100px;float:left; margin-right:15px"/>
<br /> 
##  Mini-Challenge - 2
***
### Instructions
### Decision Tree
* Fit a Decision tree model on the above data and predict on the test data and calculate the r2 score

In [37]:
from sklearn.tree import DecisionTreeRegressor
des_regressor = DecisionTreeRegressor(random_state=9)
des_regressor.fit(X_train,y_train)
y_pred_des = des_regressor.predict(X_test)
print("r_square score: ", r2_score(y_test,y_pred_des))

r_square score:  0.7255076770678552


<img src="../images/icon/ppt-icons.png" alt="Concept-Alert" style="width: 100px;float:left; margin-right:15px"/>
<br /> 

##  Mini-Challenge - 3
***
### Instructions
### Bagging
* Fit a Bagging model(i.e resampling with replacement) on the above data and predict on the test data and calculate the r2 score.Pass the parameter `base_estimator= DecisionTreeRegressor()`, `n_estimators=20` , `max_samples=100` and `random_state=0`

In [38]:
from sklearn.tree import DecisionTreeRegressor
bag_regressor = BaggingRegressor(base_estimator= DecisionTreeRegressor(), n_estimators=100 , max_samples=100 , random_state=9)
bag_regressor.fit(X_train,y_train)
y_pred_bag = bag_regressor.predict(X_test)
print("r_square score: ", r2_score(y_test,y_pred_bag))

r_square score:  0.8074589004457653


<img src="../images/icon/ppt-icons.png" alt="Concept-Alert" style="width: 100px;float:left; margin-right:15px"/>
<br /> 

##  Mini-Challenge - 4
***
### Instructions
### Pasting
* Fit a Bagging model(i.e resampling without replacement) on the above data and predict on the test data and calculate the r2 score. Pass the parameter `base_estimator= DecisionTreeRegressor`, `n_estimators=100` , `max_samples=100`, `bootstrap=False` and `random_state=0`

In [39]:
from sklearn.tree import DecisionTreeRegressor
bag_regressor_pa = BaggingRegressor(base_estimator= DecisionTreeRegressor(), n_estimators=100 , max_samples=100 ,bootstrap=False, random_state=9)
bag_regressor_pa.fit(X_train,y_train)
y_pred_bag_pa = bag_regressor_pa.predict(X_test)
print("r_square score: ", r2_score(y_test,y_pred_bag_pa))

r_square score:  0.8136994502228827


<img src="../images/icon/ppt-icons.png" alt="Concept-Alert" style="width: 100px;float:left; margin-right:15px"/>
<br /> 
##  Mini-Challenge - 5
***
### Instructions
### Random Forest
* Fit a Random Forest regressor on the above data and predict on the test data and calculate the r2 score. Pass the parameter `random_state=9`

In [40]:
rf_clf = RandomForestRegressor(random_state=9)
rf_clf.fit(X_train,y_train)
rf_y_pred =  rf_clf.predict(X_test)
print("r_square score: ", r2_score(y_test,rf_y_pred))

r_square score:  0.806088657751362


<img src="../images/icon/ppt-icons.png" alt="Concept-Alert" style="width: 100px;float:left; margin-right:15px"/>
<br /> 

##  Mini-Challenge - 6
***
### Instructions
### Random Forest tuned with GridSearchCV
* Fit a RandomForest Regressor with `random_state=9` and tune the parameters using GridSearchCV. Search for the best set of features amongst the following list of features and their value.`'n_estimators': [10,20,30],'max_depth': [6,8],'min_samples_split': [6,8]`

In [41]:
#RandomForestRegressor tuned using GridSearchCV
regressor = RandomForestRegressor(random_state=9)
params = {
            'n_estimators': [10,20,30],
            'min_samples_split': [2,4,6]
             }

search = GridSearchCV(regressor,param_grid=params)
search.fit(X_train,y_train)

y_pred_grid = search.predict(X_test)
print("r_square score: ", r2_score(y_test,y_pred_grid))

r_square score:  0.8196488360574541


<img src="../images/icon/ppt-icons.png" alt="Concept-Alert" style="width: 100px;float:left; margin-right:15px"/>
<br /> 

##  Mini-Challenge - 7
***
### Instructions
### Stacking
* Stack the below models(i.e `regressor1,regressor2,regressor3,regressor4`) and predict on the test data.
* Use the Stacking() from mlxtend to initialize a stacking Regressor object. Pass the `regressor_list` to parameter `regressors` and `lin_reg` as `meta_regressor` parameter , while initializing the object.

In [54]:
lin_reg = LinearRegression()


regressor1 = DecisionTreeRegressor(random_state=0)
regressor2= DecisionTreeRegressor(random_state=1)
regressor3 = DecisionTreeRegressor(random_state=2)
regressor4= DecisionTreeRegressor(random_state=9)
regressor_list=[regressor1,regressor2,regressor3,regressor4]


stacking_regressor = StackingRegressor(regressors=regressor_list,meta_regressor=lin_reg)
stacking_regressor.fit(X_train, y_train)
y_pred_stack = stacking_regressor.predict(X_test)
print("r_square score: ", r2_score(y_test,y_pred_stack))

r_square score:  0.7301355869242201



|Model|R2 Score|
|-----|-----|
|Base Decision Tree |0.73|
|Bagging with Replacement|0.81|
|Bagging without Replacement(pasting)|0.81|
|Random Forest|0.81|
|Random Forest with GridSearchCV|0.82|
|Stacking|0.73|